In [1]:
import numpy as np
import json
import pandas as pd


from matrix_class import ProteinMatrix
from matrix_class import SubMatrix
from cluster_class import AllClusters
from degreelist_class import DegreeList


from connected_components_utils import *
from analysis_utils import *

from func_e.FUNC_E import FUNC_E # import the class
import func_e.vocabs.all as vocabs

In [2]:
dream3_cluster_dict_file = "../data/results/DREAM-3-cc/d3_5_100.json-cluster.json"
dict_of_clusters = {}
# # convert actual cluster file to a dictionary!!
with open(dream3_cluster_dict_file,"r") as cluster_dict_file:
    dict_of_clusters = json.load(cluster_dict_file)

In [3]:
matrix, clusters, degreelist = initialize_matrix_clusters_degreelist("../data/networks/DREAM_files/dream_3.txt", "../data/results/DREAM-3-cc/d3_5_100.json-cluster.json")

In [32]:
genomic_background_filepath = '../data/testing_data/protein_list.txt'
all_go_terms = vocabs.getTerms(['GO'])
term_mapping_filepath = 'term_mapping.txt'
create_term_mapping_list('../data/go-results/dream_3_go_results.tsv', term_mapping_filepath)

In [30]:
type(all_go_terms)

pandas.core.frame.DataFrame

In [5]:
og_clusters_querylist_path = 'original_querylist.txt'
clusters.print_querylist_of_clusters_to_file([*range(clusters.get_num_clusters())]
, og_clusters_querylist_path)


original_fe = FUNC_E()
original_fe.importFiles({
    'background': genomic_background_filepath, 
    'terms2features': term_mapping_filepath })
original_fe.setTerms(all_go_terms)
original_fe.setEnrichmentSettings({'ecut': 0.01})

original_fe.importFiles({'query': og_clusters_querylist_path })
original_fe.run(cluster=False)

/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [6]:
original_df = original_fe.enrichment[['Module', 'Term', 'Fishers_pvalue']].copy()
original_df['Module'] = original_df['Module'] + ' ' + original_df['Term']
original_df.drop('Term', axis=1, inplace=True)

### Now: will create different sets of proteins to add back etc

In [7]:
# VERY STRICT CUTTOFF

qualifying_clusters_1, qualifying_proteins_1 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=.90, min_components_that_protein_connects=3)

In [13]:
new_clusters_1 = AllClusters(protein_to_cluster_dict=dict_of_clusters)
print(new_clusters_1.get_cluster_proteins(152))

['SLC9A1', 'CDC6', 'IRF8', 'IGF2R', 'PDE11A', 'PKD2L1', 'PRKAR1A', 'G3BP1', 'MTCH2', 'CTDSPL', 'TRPV5', 'PTN', 'MLST8', 'PLG', 'PCSK7', 'HERC2', 'PTEN', 'RRN3', 'RHOC', 'RALBP1', 'NDFIP2', 'MKNK2', 'SMAD1', 'NEK1', 'MAPK14', 'MEP1B']


In [14]:
new_clusters_1 = modified_clusters(AllClusters(protein_to_cluster_dict=dict_of_clusters), qualifying_proteins_1)


In [15]:
print(new_clusters_1.get_cluster_proteins(152))

['SLC9A1', 'CDC6', 'IRF8', 'IGF2R', 'PDE11A', 'PKD2L1', 'PRKAR1A', 'G3BP1', 'MTCH2', 'CTDSPL', 'TRPV5', 'PTN', 'MLST8', 'PLG', 'PCSK7', 'HERC2', 'PTEN', 'RRN3', 'RHOC', 'RALBP1', 'NDFIP2', 'MKNK2', 'SMAD1', 'NEK1', 'MAPK14', 'MEP1B', 'PPM1A', 'PPP2CA', 'CDK2', 'GSK3B', 'MAPK3', 'CSNK2A1', 'PRKACA', 'SRC']


In [35]:
# from analysis_utils import print_both_querylists_to_files
# from analysis_utils import get_initialized_fe

In [ ]:
updated_clusters_querylist_path = 'new_querylist.txt'

updated_fe = get_initialized_fe(genomic_background_filepath, term_mapping_filepath, termlist = all_go_terms)


updated_fe.importFiles({'query': updated_clusters_querylist_path })
updated_fe.run(cluster=False)

In [24]:
# instead of doing qualifying_proteins_1.keys() can do the qualifying clusters, but this might yield some clusters that didn't have qualifying proteins and were therefore not updated
print_both_querylists_to_files(qualifying_proteins_1.keys(), clusters, new_clusters_1, original_query_filepath=og_clusters_querylist_path)


In [26]:
original_df = original_fe.enrichment[['Module', 'Term', 'Fishers_pvalue']].copy()
original_df['Module'] = original_df['Module'] + ' ' + original_df['Term']
original_df.drop('Term', axis=1, inplace=True)
original_df

,Module,Fishers_pvalue
0,cluster_0 GO:0000724,0.004074
1,cluster_0 GO:0008157,0.00765
2,cluster_0 GO:0010165,0.00765
3,cluster_1 GO:0007416,0.001082
4,cluster_100 GO:0000086,0.005169
...,...,...
400,cluster_95 GO:0043666,0.006632
401,cluster_96 GO:0005179,0.001872
402,cluster_98 GO:0007156,0.006058
403,cluster_99 GO:0018105,0.006975


In [29]:

updated_df = updated_fe.enrichment[['Module', 'Term', 'Fishers_pvalue']].copy()

updated_df['Module'] = updated_df['Module'] + ' ' + updated_df['Term']
updated_df.drop('Term', axis=1, inplace=True)
updated_df.rename(columns = {'Fishers_pvalue':'Updated_Fishers_pvalue'}, inplace = True)

updated_df

,Module,Updated_Fishers_pvalue
0,cluster_152 GO:0005739,0.004911
1,cluster_152 GO:0018105,0.000449
2,cluster_152 GO:0046827,0.001368
3,cluster_161 GO:0004707,0.009507
4,cluster_161 GO:0007173,0.006526
5,cluster_161 GO:0007265,0.008565
6,cluster_161 GO:0008286,0.009345
7,cluster_161 GO:0018105,0.000277
8,cluster_161 GO:0018107,0.001944
9,cluster_161 GO:0032436,0.004365


In [ ]:
print(f"ORIGINAL CLUSTERS TABLE:\n{original_df}")
print(f"UPDATED CLUSTERS TABLE:\n{updated_df}")

In [ ]:
results_df = pd.merge(original_df, updated_df, on=['Module'], how='outer')

In [ ]:
print(results_df)

In [ ]:
results_df.to_csv("results.txt", sep='\t')


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
results_df.plot(x="Module", y=["Fishers_pvalue", "Updated_Fishers_pvalue"], kind="bar")

In [ ]:
changes_df = (results_df.copy()).dropna(axis=0)

In [ ]:
changes_df.plot(x="Module", y=["Fishers_pvalue", "Updated_Fishers_pvalue"], kind="bar")

this graph ^ is interesting bc it shows that p-values increase (less significant). this shows that maybe we are adding proteins to the wrong clusters. it will be interesting to go back and tweak the functions and see how adding different things affects everything